In [138]:
from utils import *
import mrf
import llm
from importlib import reload
from evaluate import *
import utils
import evaluate
reload(evaluate)
import json
import os
from tabulate import tabulate

In [142]:
# list all files in all subdirectories
def list_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                yield os.path.join(root, file)

k = [i for i in list_files('./Experiments/trial/bin0')]
k

['./Experiments/trial/bin0/Rochester/Rochester_q.json',
 './Experiments/trial/bin0/Newark/Newark_q.json',
 './Experiments/trial/bin0/Columbus/Columbus_q.json',
 './Experiments/trial/bin0/BrowardCounty/BrowardCounty_q.json',
 './Experiments/trial/bin0/SanDiego/SanDiego_q.json',
 './Experiments/trial/bin0/Cambridge/Cambridge_q.json',
 './Experiments/trial/bin0/SantaClaraCounty/SantaClaraCounty_q.json',
 './Experiments/trial/bin0/ClarkCounty/ClarkCounty_q.json',
 './Experiments/trial/bin0/Dallas/Dallas_q.json',
 './Experiments/trial/bin0/LosAngeles/LosAngeles_q.json',
 './Experiments/trial/bin0/Salem/Salem_q.json',
 './Experiments/trial/bin0/SanMateoCounty/SanMateoCounty_q.json',
 './Experiments/trial/bin0/Denver/Denver_q.json',
 './Experiments/trial/bin0/JerseyCity/JerseyCity_q.json',
 './Experiments/trial/bin0/FortWorth/FortWorth_q.json',
 './Experiments/trial/bin0/Oakland/Oakland_q.json',
 './Experiments/trial/bin0/Nashville/Nashville_q.json']

In [296]:
def parse_results(path):
    js = json.load(open(path))
    ground = js['ground']
    cot = []
    zero = []
    mrf = []
    question = js['question']['Text']
    for key in js.keys():
        if key in ['question', 'ground']:
            continue
        cot.append(js[key]['cot']['probability'])
        zero.append(js[key]['zero']['probability'])
        mrf.append(js[key]['mrf']['probability'])
        query = js[key]['mrf']['query']
    return ground, cot, zero, mrf, question, query
        
def metrics(ground, res):
    return evaluate.total_variation_distance(ground, res).round(3)

def metrics2(ground, res):
    return evaluate.kl_divergence(ground, res).round(3)



In [189]:

for j in list_files('./Experiments/constraints/bin1'):
    ground, cot, zero, mrf, question, query = parse_results(j)
    if len(cot) == 0:
        continue
    print(j, question)
    print(query)
    cot = np.array(cot[0])
    zero = np.array(zero[0])
    mrf = np.array(mrf[0])
    ground = np.array(ground)
    table = [
        ["Method", "total_variation_distance", "dist"],
        ["CoT", metrics(ground, cot), cot.round(2)],
        ["mrf", metrics(ground, mrf), mrf.round(2)],
        ["ground", metrics(ground, ground), ground.round(2)],
        ["zero", metrics(ground, zero), zero.round(2)],
    ]
    table2 = [
        ["Method", "kl_divergence", "dist"],
        ["CoT", metrics2(ground, cot), cot.round(2)],
        ["mrf", metrics2(ground, mrf), mrf.round(2)],
        ["ground", metrics2(ground, ground), ground.round(2)],
        ["zero", metrics2(ground, zero), zero.round(2)],
    ]
    print(tabulate(table))
    print("#################################################")

./Experiments/0611_1819/bin1/Rochester/Rochester_q.json What is the nightly price of an Airbnb listing in Rochester, NY with 2 bedrooms?
{"Queries":[{"Target":[{"Name":"Price","Value":["P1","P2","P3","P4","P5"]}],"Condition":[{"Name":"Bedrooms","Value":["B2"]},{"Name":"Location","Value":["L3"]},{"Name":"PropertyType","Value":["PT1"]},{"Name":"Season","Value":["S2"]},{"Name":"Reviews","Value":["R1"]}]}]}
------  ------------------------  --------------------------
Method  total_variation_distance  dist
CoT     0.26                      [0.14 0.26 0.38 0.17 0.05]
mrf     0.108                     [0.04 0.22 0.58 0.15 0.02]
ground  0.0                       [0.03 0.27 0.63 0.06 0.01]
zero    0.2                       [0.05 0.2  0.5  0.2  0.05]
------  ------------------------  --------------------------
#################################################
./Experiments/0611_1819/bin1/Newark/Newark_q.json What is the price of an Airbnb listing in Newark, NJ for the months of May to August?
{"

In [159]:
for j in list_files('./Experiments/trial/bin0'):
    ground, cot, zero, mrf, question = parse_results(j)
    print(j, question)

./Experiments/trial/bin0/Rochester/Rochester_q.json What is the nightly price of an Airbnb listing in Rochester, NY with 2 bedrooms?
./Experiments/trial/bin0/Newark/Newark_q.json What is the price of an Airbnb listing in Newark, NJ for the months of May to August?
./Experiments/trial/bin0/Columbus/Columbus_q.json What is the price of an Airbnb listing with 2 beds in Columbus, OH?
./Experiments/trial/bin0/BrowardCounty/BrowardCounty_q.json What is the nightly price of an Airbnb listing in Broward County, FL that accommodates 2 guests?
./Experiments/trial/bin0/SanDiego/SanDiego_q.json What is the price of an Airbnb listing in San Diego, CA with 0 to 1 bathroom?
./Experiments/trial/bin0/Cambridge/Cambridge_q.json What is the price of an Airbnb listing in Cambridge, MA for a host who is not a superhost?
./Experiments/trial/bin0/SantaClaraCounty/SantaClaraCounty_q.json What is the price of an Airbnb listing hosted by a superhost in Santa Clara County, CA?
./Experiments/trial/bin0/ClarkCount

In [158]:
for j in list_files('./Dataset/bin0'):
    if j.endswith('q.json'):
        file = json.load(open(j))
        if 'Text' in file.keys():
            print(file['Text'])
        else:
            print('no text')

What is the nightly price of an Airbnb listing in Rochester, NY with 2 bedrooms?
What is the price of an Airbnb listing in Newark, NJ for the months of May to August?
What is the price of an Airbnb listing with 2 beds in Columbus, OH?
What is the nightly price of an Airbnb listing in Broward County, FL that accommodates 2 guests?
What is the price of an Airbnb listing in San Diego, CA with 0 to 1 bathroom?
What is the price of an Airbnb listing in Cambridge, MA for a host who is not a superhost?
What is the price of an Airbnb listing hosted by a superhost in Santa Clara County, CA?
What is the price of an Airbnb listing in Clark County, NV with no review scores available?
What is the price of an Airbnb listing in Dallas, TX with a review scores rating of 4.9 or higher?
What is the price of an Airbnb listing in Los Angeles, CA that accommodates 4 or more guests per night?
What is the price of an Airbnb listing in Salem, OR, hosted by someone who has been hosting for 1-5 years?
What is t

In [136]:
k = [i for i in list_files('./Experiments/trial/bin0')]
b = ['./Experiments/trial/bin0/LosAngeles/LosAngeles_q.json']

In [130]:
import mrf
import importlib
importlib.reload(mrf)

k = [i for i in list_files('./Experiments/trial/bin0')]


for j in b[:1]:
    print(j)
    js = json.load(open(j))
    for key in js.keys():
        if key in ['question', 'ground']:
            continue
        #print(key)
        query = js[key]['mrf']['query']
        query = json.loads(query)
        query['Queries'][0]['Condition'] = [query['Queries'][0]['Condition'][0]]
        print(query)
        test = mrf.Brute(verbose=True)
        test.from_json(vars=js[key]['mrf']['json']['Variables'], constraints=pp)
        #test.from_json(pp)
        test.update(10000)
        test.query(query=query)
        break


./Experiments/trial/bin0/LosAngeles/LosAngeles_q.json
{'Queries': [{'BatchTarget': [{'Name': 'PriceRange', 'Value': ['$0-$50', '$51-$100', '$101-$200', '$201-$500', '$501+']}], 'Condition': [{'Name': 'Bedrooms', 'Value': ['2', '3', '4+']}]}]}

Variable:  PriceRange
Value:  $0-$50, $51-$100, $101-$200, $201-$500, $501+ 


Variable:  Location
Value:  Downtown, Beachfront, Suburban, NearAttractions, Remote 


Variable:  PropertyType
Value:  Apartment, House, Studio, SharedRoom, LuxuryProperty 


Variable:  Bedrooms
Value:  Studio, 1, 2, 3, 4+ 


Variable:  Amenities
Value:  Basic, Standard, Premium 


Variable:  Seasonality
Value:  LowSeason, HighSeason 


Variable:  Rating
Value:  1-2stars, 3-4stars, 4.5-5stars 

-----------------------------------------------
Finished building
Features:  206
Constraints:  206
-----------------------------------------------
Loss:  tensor(20.0290, grad_fn=<SumBackward0>) tensor(0.9826, grad_fn=<MaxBackward1>)
torch.return_types.topk(
values=tensor([0.9826

In [344]:
reload(utils)
p = './Experiments/35/'

for j in os.listdir(p):
    # check if is folder
    if os.path.isdir(p+j):
        dat = None
        for file in os.listdir(p+j):
            if file.endswith('.csv'):
                dat = utils.DatasetQ(p+j+'/'+file)
            elif file.endswith('.json'):
                constraints = json.load(open(p+j+'/'+file))
        table = [["Constraint", "Ground", "Zero", "Joint", "Change", "Zero dist", "Joint dist"],]
        tot_change = 0
        #print(constraints['Variables'])
        for cons in tmpp['Constraints']:
            er = False
            ground = dat.marg(cons)
            pred = cons['Probability']
            if np.isnan(ground).any():
                print("Error: nan in ground", cons['Question'])
                print("#################################################")
                continue
            query = {}
            query['Queries'] = [{'Condition': cons['Condition'], 'Target': cons['Target']}]
            after = mrfff.query(query)
            ground = np.array(ground)
            zero = np.array(pred)
            after = np.array(after)
            tar_name = cons['Target'][0]['Name']
            cons_val = '[]'
            if len(cons['Condition']) > 0:
                cons_val = cons['Condition'][0]['Value']
            try:
                zero_metric = metrics2(ground, zero)
            except:
                er = True
                zero_metric = float('nan')
            after_metric = metrics2(ground, after)
            table.append([f"{tar_name}|{cons_val}", str(ground.round(2)), str(zero.round(2)), str(after.round(2)), f"{(after_metric-zero_metric):+.3f}", zero_metric, after_metric])
            if not er:
                tot_change += after_metric-zero_metric
        print(tabulate(table))
        print("#################################################")
        print(f"Total change: {tot_change}")


.DS_Store
BrowardCounty
LosAngeles


In [390]:
dat = None
pa = './Experiments/35/BrowardCounty'
for file in os.listdir(pa):
    if file.endswith('.csv'):
        dat = utils.DatasetQ(pa+'/'+file)
    elif file.endswith('.json'):
        constraints = json.load(open(pa+'/'+file))
table = [["Constraint", "Ground", "Zero", "Joint", "Change", "Zero dist", "Joint dist"],]
tot_change = 0
con = remove_vars(constraints, ['Month'])
#print(constraints['Variables'])
for cons in con['Constraints']:
    er = False
    ground = dat.marg(cons)
    pred = cons['Probability']
    if np.isnan(ground).any():
        print("Error: nan in ground", cons)
        print("#################################################")
        continue
    query = {}
    query['Queries'] = [{'Condition': cons['Condition'], 'Target': cons['Target']}]
    after = model.query(query)
    ground = np.array(ground)
    zero = np.array(pred)
    after = np.array(after)
    tar_name = cons['Target'][0]['Name']
    cons_val = '[]'
    if len(cons['Condition']) > 0:
        cons_val = cons['Condition'][0]['Value']
    try:
        zero_metric = metrics(ground, zero)
    except:
        er = True
        zero_metric = float('nan')
    after_metric = metrics(ground, after)
    table.append([f"{tar_name}|{cons_val}", str(ground.round(2)), str(zero.round(2)), str(after.round(2)), f"{(after_metric-zero_metric):+.3f}", zero_metric, after_metric])
    if not er:
        tot_change += after_metric-zero_metric
print(tabulate(table))
print("#################################################")
print(f"Total change: {tot_change}")

None
None
None
None
Error: nan in ground {'Target': [{'Name': 'Price', 'Value': ['<$50', '$51-$100', '$101-$200', '$201-$500', '$500+']}], 'Probability': [0.25, 0.5, 0.1667, 0.05, 0.0333], 'Condition': [{'Name': 'Room Type', 'Value': ['Shared room']}]}
#################################################
-----------------------------------  --------------------------  --------------------------  --------------------------  ------  ---------  ----------
Constraint                           Ground                      Zero                        Joint                       Change  Zero dist  Joint dist
Number of Bedrooms|[]                [0.39 0.28 0.18 0.15]       [0.3 0.4 0.2 0.1]           [0.31 0.4  0.21 0.09]       +0.007  0.139      0.146
Number of Bathrooms|[]               [0.56 0.32 0.12]            [0.4  0.35 0.25]            [0.39 0.35 0.25]            +0.007  0.163      0.17
Host Experience|[]                   [0.09 0.34 0.5  0.07]       [0.2  0.5  0.25 0.05]       [0.19 0.52 

In [373]:
con

{'Variables': [{'Name': 'Number of Bedrooms',
   'Value': ['0/1 bedroom', '2 bedrooms', '3 bedrooms', '4+ bedrooms']},
  {'Name': 'Number of Bathrooms',
   'Value': ['0/1 bathroom', '2 bathrooms', '3+ bathrooms']},
  {'Name': 'Host Experience',
   'Value': ['<1 year', '1-5 years', '5-10 years', '10+ years']},
  {'Name': 'Superhost', 'Value': ['Yes', 'No']},
  {'Name': 'Room Type',
   'Value': ['Entire home/apt', 'Private room', 'Shared room']},
  {'Name': 'Review Score', 'Value': ['<4.3', '4.3-4.8', '4.9+']},
  {'Name': 'Price',
   'Value': ['<$50', '$51-$100', '$101-$200', '$201-$500', '$500+']}],
 'Constraints': [{'Target': [{'Name': 'Number of Bedrooms',
     'Value': ['0/1 bedroom', '2 bedrooms', '3 bedrooms', '4+ bedrooms']}],
   'Probability': [0.3, 0.4, 0.2, 0.1],
   'Condition': []},
  {'Target': [{'Name': 'Number of Bathrooms',
     'Value': ['0/1 bathroom', '2 bathrooms', '3+ bathrooms']}],
   'Probability': [0.5, 0.3, 0.2],
   'Condition': []},
  {'Target': [{'Name': 'Host E

In [389]:
import mrf
reload(mrf)

model = mrf.Brute(verbose=True)
ret = model.from_json(tmp)
model.update(10000)


Variable:  Number of Bedrooms
Value:  0/1 bedroom, 2 bedrooms, 3 bedrooms, 4+ bedrooms 


Variable:  Number of Bathrooms
Value:  0/1 bathroom, 2 bathrooms, 3+ bathrooms 


Variable:  Host Experience
Value:  <1 year, 1-5 years, 5-10 years, 10+ years 


Variable:  Superhost
Value:  Yes, No 


Variable:  Room Type
Value:  Entire home/apt, Private room, Shared room 


Variable:  Review Score
Value:  <4.3, 4.3-4.8, 4.9+ 


Variable:  Price
Value:  <$50, $51-$100, $101-$200, $201-$500, $500+ 

-----------------------------------------------
Finished building
Features:  106
Constraints:  105
-----------------------------------------------


Training Progress:   0%|          | 34/10000 [00:00<00:31, 320.67it/s]

Loss:  tensor(8.8359, grad_fn=<SumBackward0>) torch.return_types.topk(
values=tensor([0.6293, 0.6230, 0.5993, 0.5751, 0.5718], grad_fn=<TopkBackward0>),
indices=tensor([41, 75, 81, 45, 86]))


Training Progress:  11%|█         | 1093/10000 [00:02<00:25, 350.22it/s]

Loss:  tensor(0.1015, grad_fn=<SumBackward0>) torch.return_types.topk(
values=tensor([0.0907, 0.0891, 0.0585, 0.0532, 0.0463], grad_fn=<TopkBackward0>),
indices=tensor([ 0, 13,  1, 62, 37]))


...

Loss:  tensor(0.0118, grad_fn=<SumBackward0>) torch.return_types.topk(
values=tensor([0.0326, 0.0303, 0.0219, 0.0204, 0.0184], grad_fn=<TopkBackward0>),
indices=tensor([15, 18, 93, 37, 98]))


Training Progress:  91%|█████████ | 9102/10000 [00:14<00:01, 756.60it/s]

Loss:  tensor(0.0110, grad_fn=<SumBackward0>) torch.return_types.topk(
values=tensor([0.0316, 0.0303, 0.0221, 0.0190, 0.0176], grad_fn=<TopkBackward0>),
indices=tensor([15, 18, 93, 98, 37]))


Training Progress: 100%|██████████| 10000/10000 [00:15<00:00, 640.71it/s]

Loss:  tensor(0.0105, grad_fn=<SumBackward0>) torch.return_types.topk(
values=tensor([0.0307, 0.0303, 0.0223, 0.0195, 0.0172], grad_fn=<TopkBackward0>),
indices=tensor([15, 18, 93, 98,  8]))


In [387]:
tmp = con.copy()
tmp = remove_vars(tmp, [], "Price")
tmp

{'Variables': [{'Name': 'Number of Bedrooms',
   'Value': ['0/1 bedroom', '2 bedrooms', '3 bedrooms', '4+ bedrooms']},
  {'Name': 'Number of Bathrooms',
   'Value': ['0/1 bathroom', '2 bathrooms', '3+ bathrooms']},
  {'Name': 'Host Experience',
   'Value': ['<1 year', '1-5 years', '5-10 years', '10+ years']},
  {'Name': 'Superhost', 'Value': ['Yes', 'No']},
  {'Name': 'Room Type',
   'Value': ['Entire home/apt', 'Private room', 'Shared room']},
  {'Name': 'Review Score', 'Value': ['<4.3', '4.3-4.8', '4.9+']},
  {'Name': 'Price',
   'Value': ['<$50', '$51-$100', '$101-$200', '$201-$500', '$500+']}],
 'Constraints': [{'Target': [{'Name': 'Number of Bedrooms',
     'Value': ['0/1 bedroom', '2 bedrooms', '3 bedrooms', '4+ bedrooms']}],
   'Probability': [0.3, 0.4, 0.2, 0.1],
   'Condition': []},
  {'Target': [{'Name': 'Number of Bathrooms',
     'Value': ['0/1 bathroom', '2 bathrooms', '3+ bathrooms']}],
   'Probability': [0.4, 0.35, 0.25],
   'Condition': []},
  {'Target': [{'Name': 'Host

In [376]:
con['Constraints']

[{'Target': [{'Name': 'Number of Bedrooms',
    'Value': ['0/1 bedroom', '2 bedrooms', '3 bedrooms', '4+ bedrooms']}],
  'Probability': [0.3, 0.4, 0.2, 0.1],
  'Condition': []},
 {'Target': [{'Name': 'Number of Bathrooms',
    'Value': ['0/1 bathroom', '2 bathrooms', '3+ bathrooms']}],
  'Probability': [0.5, 0.3, 0.2],
  'Condition': []},
 {'Target': [{'Name': 'Host Experience',
    'Value': ['<1 year', '1-5 years', '5-10 years', '10+ years']}],
  'Probability': [0.2727, 0.4545, 0.1818, 0.0909],
  'Condition': []},
 {'Target': [{'Name': 'Superhost', 'Value': ['Yes', 'No']}],
  'Probability': [0.3, 0.7],
  'Condition': []},
 {'Target': [{'Name': 'Room Type',
    'Value': ['Entire home/apt', 'Private room', 'Shared room']}],
  'Probability': [0.6, 0.35, 0.05],
  'Condition': []},
 {'Target': [{'Name': 'Review Score', 'Value': ['<4.3', '4.3-4.8', '4.9+']}],
  'Probability': [0.15, 0.6, 0.25],
  'Condition': []},
 {'Target': [{'Name': 'Price',
    'Value': ['<$50', '$51-$100', '$101-$200',

In [380]:
def remove_vars(constraints, vars, one=''):
    ret = {}
    ret['Variables'] = []
    ret['Constraints'] = []
    check = set()
    for var in constraints['Variables']:
        if var['Name'] not in vars:
            ret['Variables'].append(var)
    for cons in constraints['Constraints']:
        if cons is None:
            print("None")
            continue
        if cons['Target'][0]['Name'] not in vars and (len(cons['Condition']) == 0 or (cons['Condition'][0]['Name'] not in vars)):
            if len(cons['Condition']) != 0 or cons['Target'][0]['Name'] != one:
                tt = {}
                tt['Target'] = cons['Target']
                tt['Probability'] = cons['Probability']
                #tt['Question'] = cons['Question']
                tt['Condition'] = []
                for cond in cons['Condition']:
                    if cond['Name'] == 'City':
                        continue
                    tt['Condition'].append(cond)
                ret['Constraints'].append(tt)
    return ret